In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('input/train.csv')

In [4]:
df = df.drop('id', axis=1)

In [5]:
maps_of_feature = { 'binary': [], 'nominal': [], 'interval': [], 'ordinal': []}
for f, i in zip(df.columns, range(len(df.columns))):
    # Defining the level
    if 'bin' in f:
        maps_of_feature['binary'].append(i-1)
    elif f == 'target':
        print(f, i)
    elif 'cat' in f or f == 'id':
        maps_of_feature['nominal'].append(i-1)
    elif df[f].dtype == float:
        maps_of_feature['interval'].append(i-1)
    elif df[f].dtype == int:
        maps_of_feature['ordinal'].append(i-1)

target 0


In [6]:
train = df.drop('target', axis=1).as_matrix()
target = df['target'].as_matrix()

In [7]:
train[train == -1] = np.nan

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

class SelectionFeature(BaseEstimator, TransformerMixin):
    
    def __init__(self, index):
        self.index = index
    
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        return X[:, self.index]
    

In [9]:
binary_select = SelectionFeature(maps_of_feature['binary'])
category_select = SelectionFeature(maps_of_feature['nominal'])

In [10]:
from sklearn.preprocessing import Imputer

In [11]:
imputer = Imputer(strategy='most_frequent')

In [12]:
binary = binary_select.fit_transform(train, target)
category = category_select.fit_transform(train, target)

In [13]:
np.sum(np.isnan(category))

696060

In [14]:
replace_missing_bin = imputer.fit_transform(binary)

In [15]:
replace_missing_cat = imputer.fit_transform(category)

In [16]:
replace_missing_cat.shape

(595212, 14)

In [17]:
from sklearn.preprocessing import OneHotEncoder

In [18]:
onehot = OneHotEncoder()
onehot_cat = onehot.fit_transform(replace_missing_cat)

In [19]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [50]:
class AutoEncoder(nn.Module):
    def __init__(self, input_size):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(True), 
            nn.Linear(64, 32))
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(True),
            nn.Linear(64, input_size),
            nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

learning_rate = 0.001

In [51]:
num_epochs = 200

In [52]:
from torch import FloatTensor

In [53]:
x_torch = Variable(FloatTensor(onehot_cat.toarray()))

In [56]:
learning_rate = 0.01
model = AutoEncoder(onehot_cat.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)


In [58]:
for epoch in range(num_epochs):
    # ===================forward=====================
    output = model(x_torch)
    loss = criterion(output, x_torch)
    # ===================backward====================
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # ===================log========================
    print(loss.data.numpy())

[ 0.05979379]
[ 0.04740255]
[ 0.04196391]
[ 0.03784223]
[ 0.03377702]


KeyboardInterrupt: 